In [62]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [63]:
dataset = pd.read_csv('Data.csv')

In [64]:
dataset.head()


,ack,ack_A,ack_B,bytes,bytes_A,bytes_A_B_ratio,bytes_B,ds_field_B,duration,http_dom_host_alexaRank,http_inter_arrivel_avg,http_inter_arrivel_entropy,http_inter_arrivel_firstQ,http_inter_arrivel_max,http_inter_arrivel_median,http_inter_arrivel_min,http_inter_arrivel_stdev,http_inter_arrivel_sum,http_inter_arrivel_thirdQ,packet_inter_arrivel_A_avg,packet_inter_arrivel_A_entropy,packet_inter_arrivel_A_firstQ,packet_inter_arrivel_A_max,packet_inter_arrivel_A_median,packet_inter_arrivel_A_min,packet_inter_arrivel_A_stdev,packet_inter_arrivel_A_sum,packet_inter_arrivel_A_thirdQ,packet_inter_arrivel_A_var,packet_inter_arrivel_B_avg,packet_inter_arrivel_B_entropy,packet_inter_arrivel_B_firstQ,packet_inter_arrivel_B_max,packet_inter_arrivel_B_median,packet_inter_arrivel_B_min,packet_inter_arrivel_B_stdev,packet_inter_arrivel_B_sum,packet_inter_arrivel_B_thirdQ,packet_inter_arrivel_B_var,packet_inter_arrivel_avg,...,packet_size_stdev,packet_size_sum,packet_size_thirdQ,packet_size_var,packets,packets_A,packets_A_B_ratio,packets_B,ssl_dom_server_name_alexaRank,ttl_A_avg,ttl_A_entropy,ttl_A_firstQ,ttl_A_max,ttl_A_median,ttl_A_min,ttl_A_stdev,ttl_A_sum,ttl_A_thirdQ,ttl_A_var,ttl_B_avg,ttl_B_entropy,ttl_B_firstQ,ttl_B_max,ttl_B_median,ttl_B_min,ttl_B_stdev,ttl_B_sum,ttl_B_thirdQ,ttl_B_var,ttl_avg,ttl_entropy,ttl_firstQ,ttl_max,ttl_median,ttl_min,ttl_stdev,ttl_sum,ttl_thirdQ,ttl_var,device_category
0,9,5,5,1213,743,0.713924,668,0,1.5756,2147483647,648.797378,0.0,366.095591,1547.304718,490.697626,218.634327,0.0,2840.981562,694.86998,4.817936,4.754888,3.562982e-01,10.584173,0.501178,0.000000e+00,4.625940,7063.094895,0.538112,21.399323,0.223995,2.0,0.067095,0.342990,1.660266e-01,0.000000e+00,0.115842,0.551958,0.244938,0.013419,3.122500,...,103.577739,679,60.5,10728.348010,10,5,0.542604,5,2147483647,64,0,64,64,64,64,0,7140,64,0,44.0,0.000000,44.0,44,44.0,44,0.000000,512,44.0,0.000000,56.965091,0.935617,44.0,64,64.0,44,9.552411,7140,64.0,91.248560,security_camera
1,9,5,5,1213,743,1.806874,668,0,0.6890,2147483647,648.797378,0.0,366.095591,1547.304718,490.697626,218.634327,0.0,2840.981562,694.86998,0.043064,3.500000,9.536743e-07,0.195949,0.011466,0.000000e+00,0.057044,0.689025,0.085983,0.003254,0.046643,2.0,0.000000,0.286143,9.536743e-07,0.000000e+00,0.085642,0.551958,0.088185,0.007335,0.022967,...,167.211532,679,60.5,27959.696429,10,5,0.823481,5,139,64,0,64,64,64,64,0,1088,64,0,129.8,0.371232,107.0,235,108.0,42,51.581420,512,229.0,2660.642857,135.000000,1.160887,64.0,235,64.0,42,79.310151,4185,235.0,6290.100000,security_camera
2,9,5,5,1213,743,0.103124,668,0,0.9852,2147483647,648.797378,0.0,366.095591,1547.304718,490.697626,218.634327,0.0,2840.981562,694.86998,0.020524,4.754888,1.579523e-03,0.121923,0.001921,9.536743e-07,0.036243,0.985172,0.013639,0.001314,0.032042,2.0,0.002094,0.196619,3.602386e-03,9.536743e-07,0.052939,0.551958,0.041513,0.002802,0.012794,...,167.211532,679,60.5,27959.696429,10,5,0.591825,5,139,64,0,64,64,64,64,0,3136,64,0,129.8,0.216397,107.0,235,108.0,43,35.653505,512,229.0,1271.172414,125.115385,1.032496,64.0,235,64.0,43,79.310151,7140,235.0,6290.100000,security_camera
3,9,5,5,1213,743,1.806874,668,0,1.5756,2147483647,648.797378,0.0,366.095591,1547.304718,490.697626,218.634327,0.0,2840.981562,694.86998,0.178558,3.418296,2.477586e-03,0.425632,0.117489,0.000000e+00,0.183261,2.142692,0.361198,0.033585,0.161996,2.0,0.000002,0.342990,3.484964e-03,0.000000e+00,0.115842,0.551958,0.244938,0.013419,0.089279,...,167.211532,679,60.5,27959.696429,10,5,0.833287,5,2147483647,64,0,64,64,64,64,0,832,64,0,42.0,0.000000,42.0,42,42.0,42,0.000000,504,42.0,0.000000,53.440000,0.998846,42.0,64,64.0,42,11.217843,1336,64.0,125.840000,security_camera
4,9,5,5,1213,743,1.806874,668,0,1.5756,2147483647,648.797378,0.0,366.095591,1547.304718,490.697626,218.634327,0.0,2840.981562,694.86998,0.170562,3.584963,3.391683e-03,0.406019,0.061885,0.000000e+00,0.188883,2.046741,0.373967,0.035677,0.208749,2.0,0.000001,0.342990,1.660266e-01,0.000000e+00,0.115842,0.551958

In [65]:
X =  dataset.iloc[:, :-1]
y = dataset.iloc[:, -1] # the first : is for the rows

In [66]:
X,y

(      ack  ack_A  ack_B   bytes  ...  ttl_stdev  ttl_sum  ttl_thirdQ      ttl_var
 0       9      5      5    1213  ...   9.552411     7140        64.0    91.248560
 1       9      5      5    1213  ...  79.310151     4185       235.0  6290.100000
 2       9      5      5    1213  ...  79.310151     7140       235.0  6290.100000
 3       9      5      5    1213  ...  11.217843     1336        64.0   125.840000
 4       9      5      5    1213  ...  11.071198     1210        64.0   122.571429
 ...   ...    ...    ...     ...  ...        ...      ...         ...          ...
 1895  264    116    148  212053  ...   5.494727    15326        64.0    30.192024
 1896   24     13     11    7749  ...   5.349335     1538        64.0    28.615385
 1897   20      9     11    7091  ...   6.251753     1261        64.0    39.084416
 1898   30     14     16    7882  ...   5.789312     1802        64.0    33.516129
 1899  294    147    147  209972  ...   7.255533    16753        64.0    52.642753
 
 [

In [67]:
X.shape, y.shape

((1900, 113), (1900,))

## Analysing the data

In [68]:
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [69]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Columns: 114 entries, ack to device_category
dtypes: float64(77), int64(36), object(1)
memory usage: 1.7+ MB


In [70]:
# checking for missing data

print("Are there missing data? :", dataset.isnull().any().any())

Are there missing data? : False


In [71]:
print("Sum of missing values:\n", dataset.isnull().sum())

Sum of missing values:
 ack                0
ack_A              0
ack_B              0
bytes              0
bytes_A            0
                  ..
ttl_stdev          0
ttl_sum            0
ttl_thirdQ         0
ttl_var            0
device_category    0
Length: 114, dtype: int64


In [72]:
# running describe()

dataset.describe().T.style.bar(subset=['mean'], color='#205ff2').background_gradient(subset=['std'], cmap='Reds').background_gradient(subset=['50%'], cmap='coolwarm')

# the range of data is pretty diverse

,count,mean,std,min,25%,50%,75%,max
ack,1900.000000,227.773158,4461.164912,0.000000,0.000000,9.000000,15.000000,184378.000000
ack_A,1900.000000,81.211579,1125.546944,0.000000,0.000000,5.000000,7.000000,39265.000000
ack_B,1900.000000,146.878947,3415.513067,0.000000,0.000000,5.000000,8.000000,145113.000000
bytes,1900.000000,149868.166842,4937287.331175,60.000000,240.000000,1213.000000,2411.000000,213714584.000000
bytes_A,1900.000000,142106.884211,4870775.723340,0.000000,0.000000,743.000000,1160.000000,210863813.000000
bytes_A_B_ratio,1900.000000,1.181570,3.477177,0.000000,0.000000,0.622177,1.201825,73.967293
bytes_B,1900.000000,7824.498947,77689.753722,60.000000,240.000000,668.000000,1096.000000,2850771.000000
ds_field_B,1900.000000,16.951579,28.199316,0.000000,0.000000,0.000000,64.000000,64.000000
duration,1900.000000,267.929434,3157.437208,0.000000,1.487675,1.575600,1.630525,58432.650400
http_dom_host_alexaRank,1900.000000,2130529862.127895,190103903.222121,1.000000,2147483647.000000,2147483647.000000,2147483647.000000,2147483647.000000


I think the problem with this is how can we visualise the data, there are a 113 columns in total. If you have an intuition about which columns especially do matter then we can try and find visualisation based on that data.

In [73]:
# total discrete features

discrete_features=[]
for col in dataset.columns[:-1]: # excluding the last columns
  if np.array_equal(dataset[col].values, dataset[col].values.astype(int)):
    discrete_features.append(col)

print(f'Total discrete features {len(discrete_features)} : ')
print(discrete_features)

Total discrete features 36 : 
['ack', 'ack_A', 'ack_B', 'bytes', 'bytes_A', 'bytes_B', 'ds_field_B', 'http_dom_host_alexaRank', 'packet_size_A_max', 'packet_size_A_sum', 'packet_size_B_max', 'packet_size_B_min', 'packet_size_B_sum', 'packet_size_max', 'packet_size_min', 'packet_size_sum', 'packets', 'packets_A', 'packets_B', 'ssl_dom_server_name_alexaRank', 'ttl_A_avg', 'ttl_A_entropy', 'ttl_A_firstQ', 'ttl_A_max', 'ttl_A_median', 'ttl_A_min', 'ttl_A_stdev', 'ttl_A_sum', 'ttl_A_thirdQ', 'ttl_A_var', 'ttl_B_max', 'ttl_B_min', 'ttl_B_sum', 'ttl_max', 'ttl_min', 'ttl_sum']


In [74]:
# lets look at all the discreet ecolumn values

for dcol in discrete_features:
  print(f'{dcol} unique value : {X[dcol].nunique()}')

ack unique value : 136
ack_A unique value : 97
ack_B unique value : 103
bytes unique value : 421
bytes_A unique value : 275
bytes_B unique value : 385
ds_field_B unique value : 4
http_dom_host_alexaRank unique value : 5
packet_size_A_max unique value : 110
packet_size_A_sum unique value : 316
packet_size_B_max unique value : 66
packet_size_B_min unique value : 1
packet_size_B_sum unique value : 220
packet_size_max unique value : 50
packet_size_min unique value : 1
packet_size_sum unique value : 358
packets unique value : 129
packets_A unique value : 103
packets_B unique value : 96
ssl_dom_server_name_alexaRank unique value : 15
ttl_A_avg unique value : 3
ttl_A_entropy unique value : 1
ttl_A_firstQ unique value : 3
ttl_A_max unique value : 3
ttl_A_median unique value : 3
ttl_A_min unique value : 3
ttl_A_stdev unique value : 1
ttl_A_sum unique value : 155
ttl_A_thirdQ unique value : 3
ttl_A_var unique value : 1
ttl_B_max unique value : 48
ttl_B_min unique value : 39
ttl_B_sum unique valu

All the discrete value columns look promising. Do you want to process them differently?

## Scaling

It is important that before running any sort of classifier we scale the data

In [75]:
X = X.values
y = y.values

In [76]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

X = ss.fit_transform(X)

In [77]:
X[:5]

array([[-4.90523791e-02, -6.77285323e-02, -4.15505067e-02,
        -3.01165989e-02, -2.90305093e-02, -1.34525340e-01,
        -9.21406316e-02, -6.01292649e-01, -8.43798110e-02,
         8.92051550e-02,  3.69722327e-01, -1.01951084e-01,
         3.60454770e-01,  3.75040773e-01,  3.61147365e-01,
         3.45001917e-01, -8.37523820e-02,  3.73080504e-01,
         3.66557519e-01, -1.85375631e-01,  9.49137376e-01,
         2.37063847e-01, -2.59280161e-01,  4.09495003e-01,
        -6.83922888e-01, -2.37421863e-01,  2.07029646e+00,
        -2.70333487e-02, -1.30196172e-01, -1.09646246e-01,
        -4.82402435e-03, -5.31046155e-02, -1.36256616e-01,
        -2.30330804e-02, -2.39695785e-01, -1.17521515e-01,
        -8.78292011e-02, -3.96231898e-02, -5.80559491e-02,
        -1.93342341e-01,  5.98413607e-01, -6.04219135e-01,
        -2.61352085e-01,  9.28795698e-01, -5.98099959e-01,
        -2.34341301e-01,  2.06940854e+00,  4.30705958e-01,
        -1.53151237e-01, -2.96784098e-01,  1.21892978e-0

## Classification

In [78]:
#split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [79]:
#Implement freature selection without label column (because label or y column is categorical)
# Use multiclass classification 
from sklearn.preprocessing import LabelBinarizer 
y_dense = LabelBinarizer().fit_transform(y) 

In [80]:
print(y_dense)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]]


In [81]:
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.svm import LinearSVC
#OneVsRestClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X)

In [82]:
#from sklearn.multiclass import OneVsOneClassifier
#from sklearn.svm import LinearSVC
#OneVsOneClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X)

In [61]:
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC
clf = OutputCodeClassifier(LinearSVC(random_state=0),code_size=2, random_state=0)
clf.fit(X, y).predict(X)

array(['security_camera', 'security_camera', 'security_camera', ...,
       'watch', 'watch', 'watch'], dtype=object)